# Log

`Pause on 20th Apr 2025`



Checked functions:

---


WUE_metrics

aerodynamic_conductance.aerodynamic_conductance(df, Rb_model = "Thom_1972")

kinematic_viscosity

reynolds_number

air_density

monin_obukhov_length

stability_parameter

stability_correction

roughness_parameters

wind_profile

```R
# Error in R code:

stability.correction <- function(zeta,formulation=c("Dyer_1970","Businger_1971")){

  formulation  <- match.arg(formulation)

  check.input(NULL,zeta)

  psi_h = psi_m <- rep(NA_real_,length(zeta))

  # universal functions
  if (formulation == "Businger_1971"){
    x_h <- -7.8
    x_m <- -6
    y_h <- 0.95 * ( 1 - 11.6 * zeta)^0.5
    y_m <- (1 - 19.3*zeta)^0.25
  } else if (formulation == "Dyer_1970"){
    x_h = x_m <- -5
    y_h       <- (1 - 16 * zeta)^0.5
    y_m       <- (1 - 16 * zeta)^0.25
  }

  # integration of universal functions (after Paulson_1970 and Foken 2008)
  # stable
  stable <- zeta >= 0 & !is.na(zeta)
  psi_h[stable] <- x_h * zeta[stable]
  psi_m[stable] <- x_m * zeta[stable]
  # unstable
  unstable <- zeta < 0 & !is.na(zeta)
  psi_h[unstable] <- 2 * log( (1 + y_h[unstable] ) / 2)
  # !!!!NOTE!!!!
  # Error here:
#   psi_m[unstable] <- 2 * log( (1 + y_m[unstable] ) / 2) +
#                      log( ( 1 + y_m[unstable]^2 ) / 2)
#                      -2 * atan(y_m[unstable]) + pi/2
  psi_m[unstable] <- (2 * log( (1 + y_m[unstable] ) / 2) +
                   log( ( 1 + y_m[unstable]^2 ) / 2)
                   -2 * atan(y_m[unstable]) + pi/2)

  return(data.frame(psi_h,psi_m))

}
```

# Test bigleaf

[tutorial](https://cran.r-project.org/web/packages/bigleaf/vignettes/bigleaf_tutorial.pdf)

[document](https://cran.r-project.org/web/packages/bigleaf/bigleaf.pdf)

Dummy data here: https://github.com/soonyenju/bigleaf/blob/main/docs/dummy_bigleaf_data.csv


Copy Python functions from https://github.com/soonyenju/bigleaf/tree/main/bigleaf here to test, and compared against R version code here: https://github.com/soonyenju/bigleaf/blob/main/bigleaf_test/R.R

Reference source: https://github.com/cran/bigleaf/tree/master/R

## Python

In [ ]:
import numpy as np
import pandas as pd
# from .bigleaf_constants import *
# from .stability_correction import stability_parameter, stability_correction
# from .meteorological_variables import kinematic_viscosity

In [ ]:
# Site parameters (adjust as needed)
zr = 40  # Reference height (m)
zh = 25  # Canopy height (m)
d = 16.75  # Displacement height (m)
z0m = 2  # Roughness length for momentum (m)
LAI = 4  # Leaf area index
leafwidth = 0.05  # Leaf width (m)
Dl = 0.05  # Leaf dimension (m)
fc = 0.8  # Fractional canopy coverage
data = pd.read_csv("dummy_bigleaf_data.csv")
data.head(1)

In [ ]:
# roughness_parameters(
#     method="wind_profile", zh=zh, zr=zr, d=d, data=data,
#     Tair='Tair', pressure='pressure', wind='wind', ustar='ustar', H='H',
#     stab_roughness=True, stab_formulation='Dyer_1970',
# )["z0m"]
# wind_profile(data, 26, zr = zr, zh = zh)

## R

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("bigleaf")
library(bigleaf)

In [ ]:
%%R

# ==============================================================================
# check_input.r
check.input <- function(data,...){

  vars <- check.length(list(...))

  if (missing(data)){
    data <- NULL
  }

  varlist  <- match.call()[-c(1:2)]
  varnames <- c(unlist(sapply(varlist,as.character)))
  varnames <- varnames[!varnames %in% c("c","list")]

  for (v in seq_along(vars)){

    var     <- vars[[v]]
    varname <- ifelse(varnames[v] %in% c("var","var_qc"),gsub("\"","",deparse(substitute(var))),varnames[v])

    if (is.character(var)){
      if (!missing(data) & !is.null(data)){
        if (length(var) == 1){
          if (var %in% colnames(data)){
            var <- data[,var]
            if (is.numeric(var)){
              assign(varname,var,pos=sys.frame(-1))
            } else {
              stop("column representing '",varname,"' in the input matrix/data.frame must be numeric",call.=FALSE)
            }
          } else {
            stop ("there is no column named '",var,"' in the input matrix/data.frame. Indicate the name of the column representing variable '",varname,"', or alternatively, provide a numeric vector of the same length as the input matrix/data.frame or of length 1.",call.=FALSE)
          }
        } else {
          stop("name of variable '",varname,"' must have length 1",call.=FALSE)
        }
      } else {
        if ("data" %in% names(formals(sys.function(which=-1)))){
          if (var %in% as.character(unlist(match.call(definition=sys.function(-1),call=sys.call(-1))[-1]))){
            stop("variable '",var,"' is of type character and interpreted as a column name, but no input matrix/data.frame is provided. Provide '",var,"' as a numeric vector, or an input matrix/data.frame with a column named '",var,"'",call.=FALSE)
          } else {
            stop("variable '",var,"' is not provided",call.=FALSE)
          }
        } else {
          stop("variable '",var,"' must be numeric",call.=FALSE)
        }
      }
    } else {
      if (length(var) < 2){
        if (is.null(var)){
          assign(varname,var,pos=sys.frame(-1))
          next
        } else if (is.na(var)){
          assign(varname,var,pos=sys.frame(-1))
          next
        }
      }
      if (!missing(data) & !is.null(data)){
        if (is.numeric(var) & length(var) == nrow(data)){
          assign(varname,var,envir=sys.frame(-1))
        } else if (is.numeric(var) & length(var) != nrow(data)) {
          if (length(var) == 1){
            var <- rep(var,length=nrow(data))
            assign(varname,var,envir=sys.frame(-1))
          } else {
            stop("variable '",varname,"' must have the same length as the input matrix/data.frame or length 1. Do NOT provide an input matrix/data.frame if none of its variables are used!",call.=FALSE)
          }
        } else if (!is.numeric(var)){
          stop("variable '",varname,"' must be numeric",call.=FALSE)
        }
      } else {
        if (is.numeric(var)){
          assign(varname,var,envir=sys.frame(-1))
        } else {
          stop("variable '",varname,"' must be numeric",call.=FALSE)
        }
      }
    }
  }
}
check.length <- function(varlist){

  if (is.list(unlist(varlist,recursive=FALSE))){
    varlist <- unlist(varlist,recursive=FALSE)
  }

  length.vars <- sapply(varlist,length)
  length.vars <- length.vars[length.vars > 0]

  if (length(unique(length.vars)) >= 2){
    if (sort(unique(length.vars))[1] != 1 | length(unique(length.vars)) > 2){
      stop("All input variables must have the same length or a length of 1!",call.=FALSE)
    }
  }
  return(varlist)
}
# ==============================================================================

stability.correction <- function(zeta,formulation=c("Dyer_1970","Businger_1971")){

  formulation  <- match.arg(formulation)

#   check.input(NULL,zeta)

  psi_h = psi_m <- rep(NA_real_,length(zeta))

  # universal functions
  if (formulation == "Businger_1971"){
    x_h <- -7.8
    x_m <- -6
    y_h <- 0.95 * ( 1 - 11.6 * zeta)^0.5
    y_m <- (1 - 19.3*zeta)^0.25
  } else if (formulation == "Dyer_1970"){
    x_h = x_m <- -5
    y_h       <- (1 - 16 * zeta)^0.5
    y_m       <- (1 - 16 * zeta)^0.25
  }

  # integration of universal functions (after Paulson_1970 and Foken 2008)
  # stable
  stable <- zeta >= 0 & !is.na(zeta)
  psi_h[stable] <- x_h * zeta[stable]
  psi_m[stable] <- x_m * zeta[stable]
  # unstable
  unstable <- zeta < 0 & !is.na(zeta)
  psi_h[unstable] <- 2 * log( (1 + y_h[unstable] ) / 2)
  psi_m[unstable] <- (2 * log( (1 + y_m[unstable] ) / 2) +
                     log( ( 1 + y_m[unstable]^2 ) / 2)
                     -2 * atan(y_m[unstable]) + pi/2) # fixed error here

  return(data.frame(psi_h,psi_m))

}

# ==============================================================================

roughness.parameters <- function(method=c("canopy_height","canopy_height&LAI","wind_profile"),zh,
                                 frac_d=0.7,frac_z0m=0.1,LAI,zr,cd=0.2,hs=0.01,data,Tair="Tair",pressure="pressure",
                                 wind="wind",ustar="ustar",H="H",d=NULL,z0m=NULL,
                                 stab_roughness=TRUE,stab_formulation=c("Dyer_1970","Businger_1971"),
                                 constants=bigleaf.constants()){

  method           <- match.arg(method)
  stab_formulation <- match.arg(stab_formulation)

  if (method == "canopy_height"){

    d      <- frac_d*zh
    z0m    <- frac_z0m*zh
    z0m_se <- NA

  } else if (method == "canopy_height&LAI"){

    X <- cd * LAI
    d <- 1.1 * zh * log(1 + X^(1/4))

    if (X >= 0 & X <= 0.2){
      z0m <- hs + 0.3 * X^(1/2)
    } else {
      z0m <- 0.3 * zh * (1 - d/zh)
    }
    z0m_se <- NA

  } else if (method == "wind_profile"){

    # check.input(data,Tair,pressure,wind,ustar,H)

    if (is.null(d)){

      d <- frac_d * zh

    }

    if (stab_roughness){

      zeta  <- stability.parameter(data=data,Tair=Tair,pressure=pressure,ustar=ustar,H=H,
                                   zr=zr,d=d,constants=constants)
      psi_m <- stability.correction(zeta,formulation=stab_formulation)[,"psi_m"]
      z0m_all <- (zr - d) * exp(-constants$k*data$wind / data$ustar - psi_m)  # fixed error here

    } else {

      z0m_all <- (zr - d) * exp(-constants$k*data$wind / data$ustar)  # fixed error here

    }

    z0m_all[z0m_all > zh] <- NA

    z0m    <- median(z0m_all,na.rm=TRUE)
    z0m_se <- constants$se_median * (sd(z0m_all,na.rm=TRUE) / sqrt(length(z0m_all[complete.cases(z0m_all)])))

  }

  return(data.frame(d,z0m,z0m_se))
}

# ==============================================================================

In [ ]:
%%R

# Site parameters (same as Python)
zr <- 40  # Reference height (m)
zh <- 25  # Canopy height (m)
d <- 16.75  # Displacement height (m)
z0m <- 2  # Roughness length for momentum (m)
LAI <- 4  # Leaf area index
leafwidth <- 0.05  # Leaf width (m)
Dl <- 0.05  # Leaf dimension (m)
fc <- 0.8  # Fractional canopy coverage


data <- read.csv("dummy_bigleaf_data.csv")
# zeta <- stability.parameter(data, zr=zr, d=d)
# stability.correction(zeta, formulation="Dyer_1970")
# wind.profile(data, 26, zr = zr, zh = zh)

In [ ]:
%%R
wind.profile <- function(data,z,Tair="Tair",pressure="pressure",ustar="ustar",H="H",wind="wind",
                         zr,zh,d=NULL,frac_d=0.7,z0m=NULL,frac_z0m=NULL,estimate_z0m=TRUE,
                         stab_correction=TRUE,stab_formulation=c("Dyer_1970","Businger_1971"),
                         constants=bigleaf.constants()){

  stab_formulation <- match.arg(stab_formulation)

  check.input(data,ustar)

  ## determine roughness parameters
  if (is.null(d)){
    if (is.null(frac_d)){
      stop("Either 'd' or 'frac_d' must be specified")
    }
    d <- frac_d * zh
  }

  if (is.null(z0m) & !estimate_z0m){
    if (is.null(frac_z0m)){
      stop("Either 'z0m' or 'frac_z0m' must be specified if 'estimate_z0m' = FALSE")
    }
    z0m <- frac_z0m * zh
  }


  if (estimate_z0m){

    if (!is.null(z0m) | !is.null(frac_z0m)){
      cat("Note that arguments 'z0m' and 'frac_z0m' are ignored if 'estimate_z0m' = TRUE. z0m is
           calculated from the logarithmic wind_profile equation.",fill=TRUE)
    }

    check.input(data,Tair,pressure,wind,ustar,H)

    z0m <- roughness.parameters(method="wind_profile",zh=zh,zr=zr,d=d,data=data,
                                Tair=Tair,pressure=pressure,wind=wind,ustar=ustar,H=H,
                                stab_roughness=TRUE,stab_formulation=stab_formulation,
                                constants=constants)[,"z0m"]
  }

  if ( any(z < (d + z0m) & !is.na(d + z0m)) ){
    warning("function is only valid for heights above d + z0m! Wind speed for heights below d + z0m will return 0!")
  }

  ## calculate wind speeds at given heights z
  if (stab_correction){

    zeta  <- stability.parameter(data=data,Tair=Tair,pressure=pressure,ustar=ustar,H=H,
                                 zr=z,d=d,constants=constants)
    psi_m <- stability.correction(zeta,formulation=stab_formulation)[,"psi_m"]
    wind_heights <- pmax(0,(ustar / constants$k) * (log(pmax(0,(z - d)) / z0m) - psi_m))

  } else {

    wind_heights <- pmax(0,(ustar / constants$k) * (log(pmax(0,(z - d)) / z0m)))

  }

  return(wind_heights)
}

# roughness.parameters(
#     method="wind_profile", zh=zh, zr=zr, d=d, data=data,
#     Tair='Tair', pressure='pressure', wind='wind', ustar='ustar', H='H',
#     stab_roughness=TRUE, stab_formulation='Dyer_1970',
# )["z0m"]

wind.profile(data, 26, zr = zr, zh = zh)

## Error Recording

In [ ]:
%%R

unstable <- -0.1
2 * log( (1 + unstable ) / 2) +
                     log( ( 1 + unstable^2 ) / 2)
                     -2 * atan(unstable) + pi/2


In [ ]:
unstable = -0.1
(2 * np.log((1 + unstable) / 2) +
                       np.log((1 + unstable**2) / 2) -
                       2 * np.arctan(unstable) + np.pi / 2)
-2 * np.arctan(unstable) + np.pi / 2